In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import matplotlib
matplotlib.use('Agg')


In [3]:
!git clone https://github.com/theAIGuysCode/yolov4-deepsort.git

fatal: destination path 'yolov4-deepsort' already exists and is not an empty directory.


In [4]:
%cd yolov4-deepsort/

/content/yolov4-deepsort


In [5]:
!ls

checkpoints	   core        object_tracker.py     save_model.py
conda-cpu.yml	   data        outputs		     tools
conda-gpu.yml	   deep_sort   README.md	     yolov4-deepsort
convert_tflite.py  LICENSE     requirements-gpu.txt
convert_trt.py	   model_data  requirements.txt


In [6]:
!pip install -r requirements-gpu.txt

In [7]:
# save yolov4-tiny model
!python save_model.py --weights ./data/yolov4-tiny.weights --output ./checkpoints/yolov4-tiny-416 --model yolov4 --tiny


2023-08-03 12:54:37.069531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-08-03 12:54:37.069607: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadd

In [8]:
# Run yolov4-tiny object tracker
!python object_tracker.py --weights ./checkpoints/yolov4-tiny-416 --model yolov4 --video ./data/video/test.mp4 --output ./outputs/tiny.avi --tiny --dont_show


Frame #:  1
FPS: 0.30
Frame #:  2
FPS: 0.96
Frame #:  3
FPS: 0.94
Frame #:  4
FPS: 2.08
Frame #:  5
FPS: 1.89
Frame #:  6
FPS: 2.05
Frame #:  7
FPS: 2.08
Frame #:  8
FPS: 2.18
Frame #:  9
FPS: 2.08
Frame #:  10
FPS: 2.06
Frame #:  11
FPS: 2.41
Frame #:  12
FPS: 2.20
Frame #:  13
FPS: 2.34
Frame #:  14
FPS: 2.48
Frame #:  15
FPS: 2.41
Frame #:  16
FPS: 1.46
Frame #:  17
FPS: 1.57
Frame #:  18
FPS: 1.51
Frame #:  19
FPS: 1.49
Frame #:  20
FPS: 1.61
Frame #:  21
FPS: 1.35
Frame #:  22
FPS: 1.46
Frame #:  23
FPS: 1.52
Frame #:  24
FPS: 1.62
Frame #:  25
FPS: 1.61
Frame #:  26
FPS: 1.74
Frame #:  27
FPS: 1.50
Frame #:  28
FPS: 1.50
Frame #:  29
FPS: 1.80
Frame #:  30
FPS: 2.60
Frame #:  31
FPS: 2.44
Frame #:  32
FPS: 2.53
Frame #:  33
FPS: 2.44
Frame #:  34
FPS: 2.54
Frame #:  35
FPS: 2.51
Frame #:  36
FPS: 2.10
Frame #:  37
FPS: 2.65
Frame #:  38
FPS: 2.60
Frame #:  39
FPS: 2.68
Frame #:  40
FPS: 2.60
Frame #:  41
FPS: 2.56
Frame #:  42
FPS: 2.46
Frame #:  43
FPS: 2.39
Frame #:  44
FPS: 2.

In [9]:
!pip install Flask requests


In [18]:
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

# YOLOv4-DeepSORT API endpoint
YOLO_DEEPSORT_API_ENDPOINT = "http://localhost:5000/track"

def count_objects_in_video(video_path):
    # Set the URL for YOLOv4-DeepSORT API
    url = YOLO_DEEPSORT_API_ENDPOINT

    # Create a multipart HTTP request with the video file
    files = {"video": open(video_path, "rb")}
    response = requests.post(url, files=files)

    # Parse the response and return the number of detected objects
    if response.status_code == 200:
        data = response.json()
        num_objects = len(data['objects'])
        return num_objects
    else:
        return None

@app.route('/count_objects', methods=['POST'])
def count_objects():
    try:
        video_file = request.files['video']
        video_path = 'uploaded_video.mp4'
        video_file.save(video_path)

        num_objects = count_objects_in_video(video_path)

        # Delete the temporary video file
        os.remove(video_path)

        if num_objects is not None:
            return jsonify({"num_objects": num_objects}), 200
        else:
            return jsonify({"error": "Failed to process the video."}), 500
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)








 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!python app.py

import requests

video_file_path = 'path_to_your_video.mp4'
files = {'video': open(video_file_path, 'rb')}

response = requests.post('http://localhost:5000/count_objects', files=files)

if response.status_code == 200:
    data = response.json()
    num_objects = data['num_objects']
    print(f'Number of objects detected: {num_objects}')
else:
    print('Failed to process the video.')